In [1]:
news_articles = [
    "삼성전자는 오늘 신제품 발표했다.",
    "현대자동차와 기아, 전기차 시장에서 선전.",
    "대통령은 경제 회복을 위한 새로운 정책 발표.",
    "한국 축구 대표팀, 월드컵에서 큰 승리.",
    "코로나 백신 접종률, 80% 돌파.",
    "오늘의 날씨, 맑음."
]

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class NewsRecommender:
    def __init__(self, news_articles):
        self.news_articles = news_articles
        self.vectorizer = TfidfVectorizer()

    def get_related_news(self, keyword, threshold=0.2):
        # 크롤링한 뉴스와 키워드를 합쳐서 TF-IDF 행렬 생성
        tfidf_matrix = self.vectorizer.fit_transform(self.news_articles + [keyword])

        # 키워드와 각 뉴스 기사 간의 cosine similarity 계산
        cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]

        # 유사도가 임계값보다 높은 뉴스만 선택
        related_news = [news for i, news in enumerate(self.news_articles) if cosine_similarities[i] > threshold]

        return related_news


# 사용법:
recommender = NewsRecommender(news_articles)
keyword = "전기차"
related_articles = recommender.get_related_news(keyword)

print(related_articles)

['현대자동차와 기아, 전기차 시장에서 선전.']


In [6]:
news_articles = [
    "삼성전자, 전기차 배터리 시장 진출 계획 발표.",
    "현대자동차와 기아, 전기차 시장 점유율 1위.",
    "전기차 인프라 확충, 정부의 새로운 녹색 정책.",
    "전기차 충전소, 전국에 1만 개가 넘게 설치될 예정.",
    "테슬라, 한국 전기차 시장에서도 강세.",
    "전기차의 미래, 자율주행과 결합될 것."
]

In [7]:
class DynamicThresholdRecommender:
    def __init__(self, news_articles, vectorizer):
        self.news_articles = news_articles
        self.vectorizer = vectorizer

    def get_related_news(self, keyword, initial_threshold=0.5, step=0.05):
        # 크롤링한 뉴스와 키워드를 합쳐서 TF-IDF 행렬 생성
        tfidf_matrix = self.vectorizer.fit_transform(self.news_articles + [keyword])

        # 키워드와 각 뉴스 기사 간의 cosine similarity 계산
        cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]

        threshold = initial_threshold
        while threshold > 0:
            # 유사도가 임계값보다 높은 뉴스만 선택
            related_news = [news for i, news in enumerate(self.news_articles) if cosine_similarities[i] > threshold]
            
            if related_news:
                return related_news, threshold
            threshold -= step
        return [], threshold

# 기존의 TfidfVectorizer 사용
vectorizer = TfidfVectorizer()

# 키워드로 "전기차 배터리"를 사용하여 관련 뉴스를 추천받는다.
recommender = DynamicThresholdRecommender(news_articles, vectorizer)
related_articles, final_threshold = recommender.get_related_news("전기차 배터리")

related_articles, final_threshold


(['삼성전자, 전기차 배터리 시장 진출 계획 발표.'], 0.4)